# TDNNS - Linear elasticity

In [ ]:
import ngsolve
print (ngsolve.__version__)

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

shape = MoveTo(0,0).Line(48,44).Line(0,16,name="right").Line(-48,-16).Close(name="fix").Face()
Draw (shape)
mesh = Mesh(OCCGeometry(shape,dim=2).GenerateMesh(maxh=5))

Draw (mesh);

In [ ]:
order=2

fesu = HCurl(mesh, order=order+1, type1=True, dirichlet="fix")
fessigma = HDivDiv(mesh, order=order, plus=True, dirichlet="default|right")
fes = fesu*fessigma

(u,sigma), (du,dsigma) = fes.TnT()
n = specialcf.normal(2)

In [ ]:
E = 70e6
nu = 1/3

# plane strain:
def Strain(stress, E,nu):
    return (1+nu)/E * stress - nu/E*Trace(stress)*Id(2)

In [ ]:
def Div(sigma,v):
    return InnerProduct(-Grad(v),sigma)*dx + InnerProduct(v*n,sigma[n,n])*dx(element_boundary=True)

term = InnerProduct(-Strain(sigma,E,nu),dsigma)*dx + Div(dsigma,u) + Div(sigma,du)
a = BilinearForm(term).Assemble()

traction = CF((0,-6.25e6))
f = LinearForm(fes)
f += traction*du.Trace()*ds("right")
f.Assemble()

gf = GridFunction(fes)
gf.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
gfu, gfsigma = gf.components

In [ ]:
Draw (gfu[1],mesh, deformation=0.2*gfu, order=5);

In [ ]:
Draw (gfsigma[0,0], mesh, min=-20e6, max=20e6);

In [ ]:
gfu(mesh(48, 60)) 

In [ ]:
fesu.ndof, fessigma.ndof

In [ ]:
mesh.GetNE(VOL), mesh.nedge, mesh.nv

In [ ]:
"order 1 dofs", 2*mesh.nedge+2*mesh.GetNE(VOL),  2*mesh.nedge+5*mesh.GetNE(VOL)

In [ ]:
"order 2 dofs", 3*mesh.nedge+6*mesh.GetNE(VOL),  3*mesh.nedge+13*mesh.GetNE(VOL)